# 3. Evolve-Filter, steady state

* $\delta = 0.1, 0.01, 0.00125, 1/nx$
* $a = 1$

In [1]:
%matplotlib inline 
import matplotlib
from dolfin import *
from __future__ import print_function
import numpy as np
import sympy as sym
import csv

x, y, t = sym.symbols('x[0], x[1], t')
sigma = 1.0
mu = 10**(-5) 
b=as_vector([2.0, 3.0])
# Exact Solution
c = 16.0 #*sym.sin(sym.pi*t)
h = x*(1-x)*y*(1-y)
g = 2*mu**(-0.5)*(0.25**2 - (x - 0.5)**2 - (y - 0.5)**2 )
iliescu = c*h*(0.5+sym.atan(g)/sym.pi)

ue = sym.simplify(iliescu)
u_code = sym.printing.ccode(ue)
u_code = u_code.replace('M_PI','DOLFIN_PI')

# du/dt - mu*Laplace(u) + div(bu) + sigma*u = f
fe = 0 #sym.diff(ue,t) Remove time dependence for now
fe += - mu*(sym.diff(sym.diff(ue,x),x) + sym.diff(sym.diff(ue,y),y))
fe += b[0]*sym.diff(ue,x) + b[1]*sym.diff(ue,y)
fe += sigma*ue

f_code = sym.printing.ccode(fe)
f_code = f_code.replace('M_PI','DOLFIN_PI')

In [2]:
# %load main-EFR.py

def compute_errors(u_e, u, t, mesh):
	L2n = errornorm(u_e, u, norm_type='L2', degree_rise=3, mesh=mesh)
	H1n = errornorm(u_e, u, norm_type='H1', degree_rise=3, mesh=mesh)
	errors = {'L2 norm': L2n, 'H1 norm': H1n}
	return 'L2, ' + str(L2n) +', H1, '+ str(H1n) +', t, '+ str(t) +'\n'

In [3]:
def evolve_filter(filename, delta, nx, u_code, f_code, sigma, mu, velocity):
    degree = 2

    u_exact = Expression(u_code, degree = degree)  
    f = Expression(f_code, degree = degree)
    mesh = UnitSquareMesh(nx,nx)
    Q = FunctionSpace(mesh, "CG", degree)

    # Set up boundary condition
    u_D = Expression(u_exact.cppcode, degree = degree)
    def boundary(x, on_boundary):
        return on_boundary
    
# ------------ STEP 1: Solve on coarse grid ----------------------#

    bc = DirichletBC(Q, u_D, boundary)

    # Test and trial functions
    u, v = TrialFunction(Q), TestFunction(Q)
    u_n = Function(Q)
    u_ = Function(Q)

    # Galerkin variational problem
    F = mu*dot(grad(v), grad(u))*dx + v*dot(velocity, grad(u))*dx + sigma*v*u*dx - f*v*dx

    # Create bilinear and linear forms
    a1 = lhs(F)
    L = rhs(F)

    # Assemble matrices
    A1 = assemble(a1)
    bc.apply(A1)

    # Create progress bar
    progress = Progress('Time-stepping')
    set_log_level(PROGRESS)

    # Step 1 Solve on Coarse Grid
    b = assemble(L)
    bc.apply(b)
    solve(A1, u_.vector(), b, 'gmres')
    
    errors = compute_errors(u_exact, u_, 0, mesh)
    print(errors)

    out_file_uexact = File("EF/evolve_steady_u_exact_"+str(nx)+".pvd") 
    ue = interpolate(u_exact, Q)
    out_file_uexact << ue
    out_file_ubar = File("EF/"+str(mu)+"evolve_steady_u_"+str(nx)+".pvd") 
    out_file_ubar << u_

    
# ------------ STEP 2: Helmholtz filter ----------------------#
    
    # Helmholtz filter to compute the indicator function
    u_tilde = TrialFunction(Q)

    #deltaH = h*vnorm/(2*mu)
    F_Hfilter = v*u_tilde*dx + delta*delta*dot(grad(v), grad(u_tilde))*dx - v*u_*dx 

    a_Hfilter = lhs(F_Hfilter)
    L_Hfilter = rhs(F_Hfilter)

    A_Hfilter = assemble(a_Hfilter)
    bc.apply(A_Hfilter)

    b_Hfilter = assemble(L_Hfilter)
    bc.apply(b_Hfilter)

    solver = LUSolver(A_Hfilter)
    u_tilde = Function(Q)
    solver.solve(u_tilde.vector(), b_Hfilter)

#     out_file_utilde << u_tilde

    # Compute the indicator function N = 0
    indicator = Expression('sqrt((a-b)*(a-b))', degree = 2, a = u_, b = u_tilde)
    indicator = interpolate(indicator, Q)
    max_ind = np.amax(indicator.vector().get_local())

    if max_ind < 1:
       max_ind = 1.0

    indicator = Expression('1.0', degree = 2, a = indicator, b = max_ind)
    indicator = interpolate(indicator, Q)

#     out_file_ind = File("EF/"+str(mu)+"EF_a_eq1_"+str(nx)+".pvd") 
#     out_file_ind << indicator

    # Apply the filter
    u_bar = TrialFunction(Q)
    F_filter = v*u_bar*dx + delta*delta*dot(grad(v), indicator*grad(u_bar))*dx - v*u_*dx 

    a_filter = lhs(F_filter)
    L_filter = rhs(F_filter)

    A_filter = assemble(a_filter)
    bc.apply(A_filter)

    b_filter = assemble(L_filter)
    bc.apply(b_filter)

    solver = LUSolver(A_filter)
    u_bar = Function(Q)
    solver.solve(u_bar.vector(), b_filter)
    
    out_file_ufilter = File("EF/delta"+filename+"_filter_u_"+str(nx)+".pvd") 
    out_file_ufilter << u_bar

    
    errors = compute_errors(u_exact, u_bar, 0, mesh)
    print(errors)


* Evolve step -- $\delta = 0.1, a = 1$

L2 = [0.575963540234, 0.263090765967, 0.041851765649,0.00554563623359,0.000221173750748,2.0995474916e-05];

H1 = [77.7640877026,66.3351391124,21.2045691927,4.4654137941,0.47857066332,0.142515173974];

* Evolve-filter -- $\delta = 0.1, a = 1$

L2 = [0.158093522563, 0.150575029186, 0.154780023975, 0.154629245194, 0.154582347719, 0];

H1 = [7.33974292022, 6.62903099806, 6.39904286939,6.37735447695, 6.37672262123, 0];

* Notes:
 * Code after filter breaks at step "solver.solve(u_tilde.vector(), b_Hfilter)"


In [4]:
delta = 0.1
filename = str(delta)
for nx in [25, 50, 100, 200, 400]:
    print('nx = '+str(nx))
    evolve_filter(filename, delta, nx, u_code, f_code, sigma, mu, b)

nx = 25
L2, 0.575963540234, H1, 77.7640877026, t, 0

L2, 0.158093522563, H1, 7.33974292022, t, 0

nx = 50
L2, 0.263090765967, H1, 66.3351391124, t, 0

L2, 0.150575029186, H1, 6.62903099806, t, 0

nx = 100
L2, 0.041851765649, H1, 21.2045691927, t, 0

L2, 0.154780023975, H1, 6.39904286939, t, 0

nx = 200
L2, 0.00554563623359, H1, 4.4654137941, t, 0

L2, 0.154629245194, H1, 6.37735447695, t, 0

nx = 400
L2, 0.000221173750748, H1, 0.47857066332, t, 0

L2, 0.154582347719, H1, 6.37672262123, t, 0



* Evolve step -- $\delta = 0.01, a = 1$
 * Same as above case with $\delta = 0.1$

* Evolve-filter -- $\delta = 0.01, a = 1$

L2 = [0.366880834568, 0.158735507535,0.0292645129898,0.0252628971399,0.0251413965989,0];

H1 = [23.7469696736, 10.3801986979,4.40346469304,4.10876151757,4.0987276568,0];


* Notes:
 * Evolve code is fine.

In [5]:
delta = 0.01
filename = str(delta)
for nx in [25, 50, 100]:
    print('nx = '+str(nx))
    evolve_filter(filename, delta, nx, u_code, f_code, sigma, mu, b)
for nx in [200, 400]:
    print('nx = '+str(nx))
    evolve_filter(filename, delta, nx, u_code, f_code, sigma, mu, b)

nx = 25
L2, 0.575963540234, H1, 77.7640877026, t, 0

L2, 0.366880834568, H1, 23.7469696736, t, 0

nx = 50
L2, 0.263090765967, H1, 66.3351391124, t, 0

L2, 0.158735507535, H1, 10.3801986979, t, 0

nx = 100
L2, 0.041851765649, H1, 21.2045691927, t, 0

L2, 0.0292645129898, H1, 4.40346469304, t, 0

nx = 200
L2, 0.00554563623359, H1, 4.4654137941, t, 0

L2, 0.0252628971399, H1, 4.10876151757, t, 0

nx = 400
L2, 0.000221173750748, H1, 0.47857066332, t, 0

L2, 0.0251413965989, H1, 4.0987276568, t, 0



* Evolve-filter -- $\delta = 0.00125, a = 1$

L2 = [0.560990349386,0.244166301512,0.0342925636421,0.00446280534452,0.0018240838882,0];

H1 = [72.8457780636,52.2460032324,11.8694635828,1.99509351811,0.772160563974,0];


In [6]:
delta = 0.00125
filename = str(delta)
for nx in [25, 50, 100]:
    print('nx = '+str(nx))
    evolve_filter(filename, delta, nx, u_code, f_code, sigma, mu, b)
for nx in [200, 400]:
    print('nx = '+str(nx))
    evolve_filter(filename, delta, nx, u_code, f_code, sigma, mu, b)

nx = 25
L2, 0.575963540234, H1, 77.7640877026, t, 0

L2, 0.560990349386, H1, 72.8457780636, t, 0

nx = 50
L2, 0.263090765967, H1, 66.3351391124, t, 0

L2, 0.244166301512, H1, 52.2460032324, t, 0

nx = 100
L2, 0.041851765649, H1, 21.2045691927, t, 0

L2, 0.0342925636421, H1, 11.8694635828, t, 0

nx = 200
L2, 0.00554563623359, H1, 4.4654137941, t, 0

L2, 0.00446280534452, H1, 1.99509351811, t, 0

nx = 400
L2, 0.000221173750748, H1, 0.47857066332, t, 0

L2, 0.0018240838882, H1, 0.772160563974, t, 0



* $\delta = 1/nx, a = 1$

L2 = [0.138924979937,0.115924927739,0.0292645129898,0.0125887591305,0.00517588091321,0];

H1 = [8.27239082645,7.21378940847,4.40346469304,2.92138500119,1.66186181004,0];

In [7]:
filename = 'varies'
for nx in [25, 50, 100, 200, 400]:
    print('nx = '+str(nx))
    delta = 1.0/nx
    evolve_filter(filename, delta, nx, u_code, f_code, sigma, mu, b)

nx = 25
L2, 0.575963540234, H1, 77.7640877026, t, 0

L2, 0.138924979937, H1, 8.27239082645, t, 0

nx = 50
L2, 0.263090765967, H1, 66.3351391124, t, 0

L2, 0.115924927739, H1, 7.21378940847, t, 0

nx = 100
L2, 0.041851765649, H1, 21.2045691927, t, 0

L2, 0.0292645129898, H1, 4.40346469304, t, 0

nx = 200
L2, 0.00554563623359, H1, 4.4654137941, t, 0

L2, 0.0125887591305, H1, 2.92138500119, t, 0

nx = 400
L2, 0.000221173750748, H1, 0.47857066332, t, 0

L2, 0.00517588091321, H1, 1.66186181004, t, 0

